## Day 22: Mode Maze

https://adventofcode.com/2018/day/22

### Part 1

Follow the instructions, creating yet another dictionary with coordinates as keys.

In [1]:
def erosion_level(depth, target_x, target_y):
    erosion = {(0, 0): 0, (target_x, target_y): 0}

    for x in range(1, target_x + 1):
        erosion[(x, 0)] = (x * 16807 + depth) % 20183
    for y in range(1, target_y + 1):
        erosion[(0, y)] = (y * 48271 + depth) % 20183

    for x in range(1, target_x + 1):
        for y in range(1, target_y + 1):
            if x != target_x or y != target_y:
                erosion[(x, y)] = (erosion[(x - 1, y)] * erosion[(x, y - 1)] + depth) % 20183
                
    return erosion

In [2]:
test_erosion = erosion_level(510, 10, 10)

In [3]:
def erosion_types(depth, max_x, max_y):
    el = erosion_level(depth, max_x, max_y)
    return {k: el[k] % 3 for k in el}

In [4]:
test_erosion_types = erosion_types(510, 10, 10)

In [5]:
sum(test_erosion_types.values())

114

In [6]:
problem_erosion = erosion_types(10689, 11, 722)

In [7]:
sum(problem_erosion.values())

8575

### Part 2

Use A\* to find the path with Manhattan distance to the target as heuristic. Valid moves are either moving to a neighbouring square where the current equipment will still be appropriate or changing equipment to another type that's appropriate for the current square. 

The erosion levels can go out of the bounds of the target so convert to a class where the values are lazily evaluated.

In [8]:
class Erosion:
    def __init__(self, depth, target_x, target_y):
        self.depth = depth
        self.erosion_level = {(0, 0): 0, (target_x, target_y): 0}
        
    def erosion_l(self, coordinates):
        if coordinates in self.erosion_level:
            return self.erosion_level[coordinates] 
        
        x, y = coordinates
        if y == 0:
            el = (x * 16807 + self.depth) % 20183
        elif x == 0:
            el = (y * 48271 + self.depth) % 20183
        else:
            el = (self.erosion_l((x - 1, y)) * self.erosion_l((x, y - 1)) 
                  + self.depth) % 20183
        self.erosion_level[coordinates] = el
        return el 
    
    def erosion(self, coordinates):
        return self.erosion_l(coordinates) % 3


e = Erosion(510, 10, 10)
sum(e.erosion((x, y)) for x in range(11) for y in range(11))

114

In [9]:
from heapq import heappop, heappush


def manhattan_neighbours(coordinates):
    x, y = coordinates
        
    for d in (-1, 1):
        if x + d >= 0:
            yield (x + d, y)
        if y + d >= 0:
            yield (x, y + d)
            
                
def manhattan_distance(coordinates_1, coordinates_2):
    x1, y1 = coordinates_1
    x2, y2 = coordinates_2
    return abs(x1 - x2) + abs(y1 - y2)


def get_path(paths, state):
    if state is None:
        return []
    else:
        return get_path(paths, paths[state]) + [state]
    
    
def quickest_path(depth, target_x, target_y):
    equipment = {0: ['torch', 'climbing gear'],
                 1: ['climbing gear', 'neither'],
                 2: ['torch', 'neither']}
    
    erosion = Erosion(depth, target_x, target_y)
    
                
    def valid_moves(state):
        coordinates, gear = state
        
        for new_gear in equipment[erosion.erosion(coordinates)]:
            if new_gear != gear:
                yield (coordinates, new_gear)
        
        for new_location in manhattan_neighbours(coordinates):
            if gear in equipment[erosion.erosion(new_location)]:
                yield (new_location, gear)
                
                
    def time_taken(from_state, to_state):
        if from_state[1] != to_state[1]:
            return 7
        else:
            return 1
                
                
    start_state = ((0, 0), 'torch')
    queue = [(manhattan_distance((0, 0), (target_x, target_y)), start_state)]
    time_to = {start_state: 0}
    # In case we want to reconstruct the path
    previous = {start_state: None}
    
    while queue:
        _, current_state = heappop(queue)
        
        for next_state in valid_moves(current_state):
            time_to_next = time_to[current_state] + time_taken(current_state, next_state)
            
            if next_state not in time_to or time_to_next < time_to[next_state]:
                time_to[next_state] = time_to_next
                previous[next_state] = current_state
                
                if next_state == ((target_x, target_y), 'torch'):
                    return time_to_next, get_path(previous, next_state)
                 
                heappush(queue, 
                         (time_to_next + manhattan_distance(next_state[0], 
                                                            (target_x, target_y)),
                          next_state))
                
    return None

In [10]:
quickest_path(510, 10, 10)

(45,
 [((0, 0), 'torch'),
  ((0, 1), 'torch'),
  ((1, 1), 'torch'),
  ((1, 1), 'neither'),
  ((2, 1), 'neither'),
  ((3, 1), 'neither'),
  ((4, 1), 'neither'),
  ((4, 1), 'climbing gear'),
  ((4, 2), 'climbing gear'),
  ((4, 3), 'climbing gear'),
  ((4, 4), 'climbing gear'),
  ((4, 5), 'climbing gear'),
  ((4, 6), 'climbing gear'),
  ((4, 7), 'climbing gear'),
  ((4, 8), 'climbing gear'),
  ((4, 9), 'climbing gear'),
  ((5, 9), 'climbing gear'),
  ((5, 10), 'climbing gear'),
  ((5, 11), 'climbing gear'),
  ((6, 11), 'climbing gear'),
  ((6, 12), 'climbing gear'),
  ((7, 12), 'climbing gear'),
  ((8, 12), 'climbing gear'),
  ((8, 11), 'climbing gear'),
  ((8, 10), 'climbing gear'),
  ((9, 10), 'climbing gear'),
  ((10, 10), 'climbing gear'),
  ((10, 10), 'torch')])

In [11]:
%time quickest_path(10689, 11, 722)

CPU times: user 3.47 s, sys: 36.3 ms, total: 3.5 s
Wall time: 3.5 s


(999,
 [((0, 0), 'torch'),
  ((0, 1), 'torch'),
  ((1, 1), 'torch'),
  ((1, 2), 'torch'),
  ((1, 3), 'torch'),
  ((2, 3), 'torch'),
  ((2, 4), 'torch'),
  ((2, 5), 'torch'),
  ((3, 5), 'torch'),
  ((3, 6), 'torch'),
  ((3, 7), 'torch'),
  ((3, 7), 'neither'),
  ((3, 8), 'neither'),
  ((3, 9), 'neither'),
  ((3, 10), 'neither'),
  ((3, 11), 'neither'),
  ((3, 12), 'neither'),
  ((3, 13), 'neither'),
  ((3, 14), 'neither'),
  ((2, 14), 'neither'),
  ((2, 15), 'neither'),
  ((2, 16), 'neither'),
  ((1, 16), 'neither'),
  ((1, 17), 'neither'),
  ((1, 18), 'neither'),
  ((1, 19), 'neither'),
  ((2, 19), 'neither'),
  ((2, 20), 'neither'),
  ((2, 21), 'neither'),
  ((2, 22), 'neither'),
  ((2, 23), 'neither'),
  ((2, 23), 'climbing gear'),
  ((2, 24), 'climbing gear'),
  ((1, 24), 'climbing gear'),
  ((1, 25), 'climbing gear'),
  ((1, 26), 'climbing gear'),
  ((1, 27), 'climbing gear'),
  ((1, 28), 'climbing gear'),
  ((1, 29), 'climbing gear'),
  ((2, 29), 'climbing gear'),
  ((2, 30), 'cli

That's a suspiciously neat number but turns out to be correct.